# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [92]:
df = data.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index()
df

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [15]:
customer_matrix = pd.pivot_table(data, values='Quantity',index=['ProductName'],columns=['CustomerID'],aggfunc=sum,fill_value=0)
customer_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [41]:
from scipy.spatial.distance import squareform, pdist
distances = pdist(customer_matrix.T, 'euclidean')
distances = pd.DataFrame(1/(1 + squareform(distances)), 
                         index=customer_matrix.columns, columns=customer_matrix.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [51]:
customerID = 23548
top5 = distances[customerID].sort_values(ascending=False)[1:6]
top5

CustomerID
7772    0.019516
412     0.019445
1534    0.019392
639     0.019370
3074    0.019333
Name: 23548, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [116]:
top5_df = df[df['CustomerID'].isin(top5.index.to_list())]
top5_df

,CustomerID,ProductName,Quantity
254,412,Apricots - Dried,1
255,412,Banana - Leaves,1
256,412,Bananas,1
257,412,Beans - Wax,1
258,412,Beef - Rib Eye Aaa,1
...,...,...,...
5561,7772,Wanton Wrap,2
5562,7772,Wine - Chardonnay South,2
5563,7772,Wine - Ej Gallo Sierra Valley,2
5564,7772,Wine - Prosecco Valdobiaddene,2


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [122]:
ranked_products = top5_df.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index().sort_values(by=['Quantity'],ascending=False)
ranked_products

,CustomerID,ProductName,Quantity
253,7772,Banana Turning,4
312,7772,"Wine - Red, Colio Cabernet",2
268,7772,"Fish - Scallops, Cold Smoked",2
263,7772,Chicken - Wieners,2
54,412,Spinach - Baby,2
...,...,...,...
106,639,Pernod,1
105,639,"Pepper - Paprika, Hungarian",1
104,639,"Peas - Pigeon, Dry",1
103,639,Pastry - Choclate Baked,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [143]:
merge_result = ranked_products.merge(customer_matrix,on='ProductName')
top5_products = merge_result[merge_result[customerID] == 0].head(5)
top5_products['ProductName'].to_list()

['Banana Turning',
 'Wine - Red, Colio Cabernet',
 'Fish - Scallops, Cold Smoked',
 'Fish - Scallops, Cold Smoked',
 'Chicken - Wieners']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [144]:
recommendations = {}
customer_list = data.CustomerID.unique()
for c in customer_list:
    top5 = distances[c].sort_values(ascending=False)[1:6]
    top5_df = df[df.CustomerID.isin(top5.index.to_list())]
    ranked_products = top5_df.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index().sort_values(by=['Quantity'],ascending=False)
    merge_result = ranked_products.merge(customer_matrix,on='ProductName')
    top5_products = merge_result[merge_result[c] == 0].head(5)
    recommendations[c] = top5_products['ProductName'].to_list()


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [160]:
pd.DataFrame.from_dict(recommendations, orient='index', columns =['Product']*5)

,Product,Product,Product,Product,Product
61288,Pasta - Orecchiette,Squid U5 - Thailand,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Lime Cordial - Roses
77352,Cocktail Napkin Blue,Chicken - Soup Base,Chicken - Soup Base,Cheese - Taleggio D.o.p.,Beef - Top Sirloin - Aaa
40094,Grouper - Fresh,Sherry - Dry,Beef - Inside Round,Halibut - Steaks,Halibut - Steaks
23548,Banana Turning,"Wine - Red, Colio Cabernet","Fish - Scallops, Cold Smoked","Fish - Scallops, Cold Smoked",Chicken - Wieners
78981,Lettuce - Frisee,Vanilla Beans,Vanilla Beans,Beef - Inside Round,Beef - Inside Round
...,...,...,...,...,...
49005,"Pasta - Penne, Rigate, Dry","Pasta - Penne, Rigate, Dry",Beets - Mini Golden,Juice - Orange,"Garlic - Primerba, Paste"
41286,Beef - Striploin Aa,"Thyme - Lemon, Fresh","Thyme - Lemon, Fresh",Bread Foccacia Whole,Bread Foccacia Whole
85878,Wine - White Cab Sauv.on,Wine - White Cab Sauv.on,Pasta - Angel Hair,Olives - Kalamata,Olives - Kalamata
68506,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls","Veal - Brisket, Provimi,bnls",Baking Powder,Flour - Teff


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [164]:
distances = pdist(customer_matrix.T, 'correlation')
distances = pd.DataFrame(1/(1 + squareform(distances)), 
                         index=customer_matrix.columns, columns=customer_matrix.columns)


recommendations = {}
customer_list = data.CustomerID.unique()
for c in customer_list:
    top5 = distances[c].sort_values(ascending=False)[1:6]
    top5_df = df[df.CustomerID.isin(top5.index.to_list())]
    ranked_products = top5_df.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index().sort_values(by=['Quantity'],ascending=False)
    merge_result = ranked_products.merge(customer_matrix,on='ProductName')
    top5_products = merge_result[merge_result[c] == 0].head(5)
    recommendations[c] = top5_products['ProductName'].to_list()

In [166]:
pd.DataFrame.from_dict(recommendations, orient='index', columns =['Product']*5)

,Product,Product,Product,Product,Product
61288,Wine - Two Oceans Cabernet,Wine - Two Oceans Cabernet,Jagermeister,Jagermeister,Jagermeister
77352,Fondant - Icing,Fondant - Icing,"Potatoes - Instant, Mashed",Bread - Rye,Lettuce - Frisee
40094,Juice - V8 Splash,"Bread - Roll, Soft White Round","Bread - Roll, Soft White Round",Beef - Inside Round,Truffle Cups - Brown
23548,Wine - Charddonnay Errazuriz,Wine - Charddonnay Errazuriz,"Coconut - Shredded, Sweet","Coconut - Shredded, Sweet",Foam Cup 6 Oz
78981,Sauerkraut,"Cup - 6oz, Foam",Cookie Chocolate Chip With,"Bar Mix - Pina Colada, 355 Ml","Tart Shells - Sweet, 4"
...,...,...,...,...,...
49005,Crush - Cream Soda,Cheese - Taleggio D.o.p.,"Pepper - White, Ground","Pepper - White, Ground",Squid - Tubes / Tenticles 10/20
41286,Bandage - Flexible Neon,Bandage - Flexible Neon,Table Cloth 62x114 White,"Mushroom - Porcini, Dry","Mushroom - Porcini, Dry"
85878,Macaroons - Two Bite Choc,Macaroons - Two Bite Choc,Apricots - Halves,Apricots - Halves,Jolt Cola - Electric Blue
68506,"Salmon - Atlantic, Skin On",Assorted Desserts,Garbage Bags - Clear,Bread Fig And Almond,Bread Fig And Almond
